In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import subprocess
from pathlib import Path
from glob import glob
import re
import os
from os.path import join as pjoin

In [8]:
dry_run = False

RAW_DIR = '/Users/richard/Desktop/TestRaw/'
OUT_DIR = '/Users/richard/Desktop/Tout/'
TRAJS_DIR = OUT_DIR
trajs_per_vid = 2

renderer_path = '../PanoRenderer/build/main'
vids_paths = sorted(glob(pjoin(RAW_DIR, '*.mp4')))
trajs_paths = sorted(glob(pjoin(TRAJS_DIR, 't*.npy')))
yc_paths = [i[:-4] + '-yc.npy' for i in vids_paths]

In [9]:
render_traj_path = 'tmp-render-traj.npy'


traj_idx = 0
for vid_path, yc_path in zip(vids_paths, yc_paths):

    #a, b, v_id, _ = parse_vid_name(vid_path)
    #tqdm.write(f'Processing video {a}-{b}-v{v_id}')

    # Sometimes OpenCV fails to read the last few frames of the video.
    # So we subtract 30 frames to be safe
    cap = cv2.VideoCapture(vid_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 30
    cap.release()

    vid_stem = Path(vid_path).stem
    print(f'Processing video {vid_stem}')

    for _ in range(trajs_per_vid):

        traj_stem = Path(trajs_paths[traj_idx]).stem
        print(f'\tUsing trajectory {traj_stem}...')

        # Load desired trajectory & apply yaw correction
        traj = np.load(trajs_paths[traj_idx])

        # Random start frame
        try:
            start_frame = np.random.randint(0, total_frames - len(traj))
        except:
            continue

        yc = np.load(yc_path)
        for i, e in enumerate(traj, start=start_frame):
            e[0] += yc[i]
        np.save(render_traj_path, traj)

        output_vid_path = pjoin(OUT_DIR, f'{traj_stem}-{vid_stem}.mp4')

        render_cmd = [
            renderer_path, vid_path, render_traj_path,
            output_vid_path, str(start_frame)]

        if dry_run:
            print('\t\t' + ' '.join(render_cmd))
        else:
            result = subprocess.run(render_cmd)
            assert result.returncode == 0

        traj_idx += 1

try:
    os.remove(render_traj_path)
except Exception as e:
    print(e)

Processing video v00
	Using trajectory t000...
	Using trajectory t001...
